In [2]:
! pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.2 MB/s eta 0:00:0000:0100:01m


In [4]:
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

data = pd.read_excel('SEWAA-data-sample.xlsx',sheet_name='gridded',index_col=0,skiprows=[1,2])
meta = pd.read_excel('SEWAA-data-sample.xlsx',sheet_name='gridded',index_col=0,nrows=2)

ds = data.to_xarray().rename({'ID':'time'})

# Not sure if this is the best way but it works
ds['time'] = np.array([t[:4]+'-'+t[4:6]+'-'+t[-2:] for t in ds['time'].values.astype(str)],dtype='datetime64[ns]')

ds = ds.assign_coords({'latitude':meta.values[0],'longitude':meta.values[1]})

ds

<xarray.Dataset> Size: 6kB
Dimensions:    (time: 365, latitude: 1, longitude: 1)
Coordinates:
  * time       (time) datetime64[ns] 3kB 2018-01-01 2018-01-02 ... 2018-12-31
  * latitude   (latitude) float64 8B 36.65
  * longitude  (longitude) float64 8B -1.34
Data variables:
    NGONG      (time) float64 3kB 2.332 4.642 10.11 7.531 ... 0.0 0.0 1.132 0.0